# Install pipelines SDK

#### Please wait till this cell completes and then run next cells. This just need to be run once per active kernel.

In [ ]:
%env RELEASE_VERSION=1.0.0
!pip install https://storage.googleapis.com/ml-pipeline/release/${RELEASE_VERSION}/kfp.tar.gz --upgrade

# Import kfp pkgs

In [ ]:
import os
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client

# List existing pipeline experiments

In [ ]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=existing_token)
client.list_experiments()

# Create a Dkube RESNETv2 experiment

In [ ]:
resnet_experiment = client.create_experiment(name='Dkube - Resnetv2 pl')

# Define e2e RESENTv2 Pipeline with Dkube components

In [ ]:
import kfp.dsl as dsl
from kfp import components
from kubernetes import client as k8s_client

import os
import json
from random import randint

dkube_training_op           = components.load_component_from_file("../components/training/component.yaml")
dkube_serving_op            = components.load_component_from_file("../components/serving/component.yaml")
dkube_viewer_op             = components.load_component_from_file('../components/viewer/component.yaml')

@dsl.pipeline(
    name='resnetv2',
    description='sample resnetv2 model based catsdogs classification pipeline with dkube components'
)
def d3pipeline(
    #Name of the project in dkube
    training_program,
    #Dataset to train on
    training_dataset,
    #Output model 
    training_output_model,
    #Dkube authentication token
    auth_token= os.getenv("DKUBE_USER_ACCESS_TOKEN"),
    #Framework. One of tensorflow, pytorch, sklearn, custom
    framework = "tensorflow",
    #Framework version
    version = '1.14',
    #By default tf v1.14 image is used here, v1.13 or v1.14 can be used. 
    #Or any other custom image name can be supplied.
    #For custom private images, please input username/password
    training_container=json.dumps({'image':'docker.io/ocdr/d3-datascience-tf-cpu:v1.14', 'username':'', 'password': ''}),
    #Script to run inside the training container
    training_script="python model.py",
    #Input dataset mount paths
    training_input_dataset_mount="/opt/dkube/input",
    #Output models for training.
    training_output_mount="/opt/dkube/output",
    #Request gpus as needed. Val 0 means no gpu, then training_container=docker.io/ocdr/dkube-datascience-tf-cpu:v1.12.
    training_gpus=0,
    #Any envs to be passed to the training program
    training_envs=json.dumps([{"steps": 100}]),
    #Device to be used for serving - dkube mnist example trained on gpu needs gpu for serving else set this param to 'cpu'
    serving_device='cpu',
    #Serving image
    serving_image=json.dumps({'image':'ocdr/tensorflowserver:1.14', 'username':'', 'password': ''}),
    #Transformer image
    transformer_image=json.dumps({'image':'docker.io/ocdr/d3-datascience-tf-cpu:v1.14', 'username':'', 'password': ''}),
    #Script to execute the transformer
    transformer_code="tf/classification/resnet/catsdogs/transformer/transformer.py"
):

    train       = dkube_training_op(auth_token, training_container,
                                    program=training_program, run_script=training_script,
                                    datasets=json.dumps([str(training_dataset)]), outputs=json.dumps([str(training_output_model)]),
                                    input_dataset_mounts=json.dumps([str(training_input_dataset_mount)]),
                                    output_mounts=json.dumps([str(training_output_mount)]),
                                    ngpus=training_gpus,
                                    envs=training_envs,
                                    framework=framework, version=version)
    serving     = dkube_serving_op(auth_token, train.outputs['artifact'],
                                device=serving_device, serving_image=serving_image,
                                transformer_image=transformer_image,
                                transformer_project=training_program,
                                transformer_code=transformer_code).after(train)

# Compile and generate tar ball

In [ ]:
compiler.Compiler().compile(d3pipeline, 'dkube_resnetv2_pl.tar.gz')
# Upload this generated tarball into the Pipelines UI

# Create and Run pipeline

#### Click the dkube-training stage to see the enhanced Dkube Datascience dashboard, metrics and graphs. Click the dkube-viewer stage for the simple UI to test the model predecitions

#### <font color=red> Fill the training_program, training_dataset and training_output_model with appropriate values in the next cell </font>

In [ ]:
run = client.run_pipeline(resnet_experiment.id, 'resnetv2_classifier_pipeline', 'dkube_resnetv2_pl.tar.gz', params={"training_program":"", "training_dataset":"", "training_output_model":""})